In [1]:
# Data processing
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.stats

# Visualization
import seaborn as sns

# Similarity
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\Manish Seal\AppData\Local\Temp\ipykernel_18192\1647266666.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
%%capture
%run item_item_collaborative_filtering.ipynb

In [3]:
matrix = pd.read_pickle("../data/item_matrix.pkl")

In [4]:
transposed_matrix = matrix.transpose()
train_transposed, test_transposed = train_test_split(transposed_matrix, test_size=0.2, random_state=42)

# Transpose the train and test sets back to the original format
train_matrix = train_transposed.transpose()
test_matrix = test_transposed.transpose()

In [35]:
# finding RMSE
N = 10

# Function to calculate RMSE for the item-item collaborative filtering model
def calculate_rmse_item(test_matrix, N):
    actual_ratings_list = []
    predicted_ratings_list = []
    for userID in test_matrix.columns:
        for songID in test_matrix.index:
            actual_rating = test_matrix.loc[songID, userID]
            if not np.isnan(actual_rating): # disregarding NA values
                actual_ratings_list.append(actual_rating)
                
                # calculate the predicted rating
                predicted_rating = get_predicted_ratings(songID, N, userID, test_matrix)
                predicted_ratings_list.append(predicted_rating)

    # Convert lists to numpy arrays for calculation
    actual_ratings_array = np.array(actual_ratings_list)
    predicted_ratings_array = np.array(predicted_ratings_list)
                
    # Calculate RMSE only for non-zero and non-NaN ratings
    non_zero_indices = np.where(actual_ratings_array != 0)[0]  # Indices of non-zero ratings
    actual_ratings_non_zero = actual_ratings_array[non_zero_indices]
    predicted_ratings_non_zero = predicted_ratings_array[non_zero_indices]

    # Calculate RMSE
    return np.sqrt(np.mean((actual_ratings_non_zero - predicted_ratings_non_zero) ** 2))

In [36]:
rmse = calculate_rmse_item(test_matrix, N)

In [26]:
print(f"RMSE for item-item collaborative filtering model: {rmse:.2f}")

RMSE for item-item collaborative filtering model: 3.69
